# A2A.MI - Algorithmic Trading Analysis

**Generated:** 2026-02-06 22:13:33

## Overview

Automated trading analysis for **A2A.MI** using the algoshort library.

### Workflow:
1. Data Acquisition from Yahoo Finance
2. Relative Price Calculation vs FTSE MIB
3. Signal Generation (Breakout, Turtle, MA Crossover, Floor/Ceiling)
4. Returns and Equity Curve Calculation
5. Stop Loss and Position Sizing


In [1]:
# =============================================================================
# CONFIGURATION
# =============================================================================
TICKER = "A2A.MI"
BENCHMARK = "FTSEMIB.MI"
START_DATE = "2016-01-01"
INITIAL_CAPITAL = 100000

# =============================================================================
# IMPORTS
# =============================================================================
import logging
import pandas as pd
import numpy as np
from datetime import date
import warnings

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.WARNING)

print(f"Analysis for: A2A.MI")
print(f"Benchmark: {BENCHMARK}")
print(f"Period: {START_DATE} to {date.today()}")

Analysis for: A2A.MI
Benchmark: FTSEMIB.MI
Period: 2016-01-01 to 2026-02-06


In [2]:
# =============================================================================
# 1. DATA ACQUISITION
# =============================================================================
from algoshort.yfinance_handler import YFinanceDataHandler

handler = YFinanceDataHandler(cache_dir="./cache", enable_logging=False)

print(f"Downloading A2A.MI and {BENCHMARK}...")
handler.download_data(
    symbols=[TICKER, BENCHMARK],
    start=START_DATE,
    end=date.today().isoformat(),
    use_cache=True
)

df = handler.get_ohlc_data(TICKER)
bmk = handler.get_ohlc_data(BENCHMARK)
df['fx'] = 1

print(f"Downloaded {len(df)} rows for A2A.MI")
print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")

/opt/hostedtoolcache/Python/3.11.14/x64/lib/python3.11/site-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


[                       0%                       ]

[*********************100%***********************]  2 of 2 completed

Downloaded 2565 rows for A2A.MI
Date range: 2016-01-04 to 2026-02-05


In [3]:
# =============================================================================
# 2. RELATIVE PRICE CALCULATION
# =============================================================================
from algoshort.ohlcprocessor import OHLCProcessor

processor = OHLCProcessor()
df = processor.calculate_relative_prices(
    stock_data=df,
    benchmark_data=bmk,
    benchmark_column='close',
    rebase=True
)

print("Relative OHLC columns created: ropen, rhigh, rlow, rclose")

Relative OHLC columns created: ropen, rhigh, rlow, rclose


In [4]:
# =============================================================================
# 3. SIGNAL GENERATION
# =============================================================================
from algoshort.regime_bo import RegimeBO
from algoshort.regime_ma import TripleMACrossoverRegime
from algoshort.regime_fc import RegimeFC

# Breakout signals
regime_bo = RegimeBO(ohlc_stock=df)
for window in [20, 50, 100]:
    df = regime_bo.compute_regime(regime_type='breakout', window=window, relative=True, inplace=True)

# Turtle Trader
df = regime_bo.compute_regime(regime_type='turtle', window=50, fast_window=20, relative=True, inplace=True)

# Moving Average Crossover
regime_ma = TripleMACrossoverRegime(ohlc_stock=df)
for ma_type in ['sma', 'ema']:
    df = regime_ma.compute_ma_regime(
        ma_type=ma_type,
        short_window=50, medium_window=100, long_window=150,
        relative=True, inplace=True
    )

# Floor/Ceiling Regime
regime_fc = RegimeFC(df=df, log_level=logging.WARNING)
df = regime_fc.compute_regime(relative=True, lvl=3, vlty_n=63, threshold=0.05, dgt=3, d_vol=1, dist_pct=0.05, retrace_pct=0.05, r_vol=1.0)

# Collect signal columns
signal_columns = [col for col in df.columns
    if any(col.startswith(p) for p in ['rbo_', 'rtt_', 'rsma_', 'rema_', 'rrg'])
    and not any(k in col for k in ['short', 'medium', 'long', '_ch'])]
signal_columns = [x for x in signal_columns if x != "rrg_ch"]

print(f"Generated {len(signal_columns)} signals: {signal_columns}")

Generated 11 signals: ['rbo_20', 'rbo_50', 'rbo_100', 'rtt_5020', 'rsma_50100', 'rsma_100150', 'rsma_50100150', 'rema_50100', 'rema_100150', 'rema_50100150', 'rrg']


In [5]:
# =============================================================================
# 4. RETURNS CALCULATION
# =============================================================================
from algoshort.returns import ReturnsCalculator

returns_calc = ReturnsCalculator(
    ohlc_stock=df,
    open_col="open", high_col="high", low_col="low", close_col="close",
    relative_prefix="r"
)

df = returns_calc.get_returns_multiple(df=df, signals=signal_columns, relative=True, n_jobs=-1)
print("Returns calculated for all signals")

Returns calculated for all signals


In [6]:
# =============================================================================
# 5. STOP LOSS CALCULATION
# =============================================================================
from algoshort.stop_loss import StopLossCalculator

sl_calc = StopLossCalculator(df)
for signal in signal_columns:
    df = sl_calc.atr_stop_loss(signal=signal, window=14, multiplier=2.0)
    sl_calc.data = df

print(f"ATR stop losses calculated for {len(signal_columns)} signals")

ATR stop losses calculated for 11 signals


In [7]:
# =============================================================================
# 6. POSITION SIZING & EQUITY CURVES
# =============================================================================
from algoshort.position_sizing import PositionSizing, run_position_sizing_parallel

sizer = PositionSizing(
    tolerance=-0.10, mn=-0.0025, mx=-0.05,
    equal_weight=0.05, avg=0.03, lot=1,
    initial_capital=INITIAL_CAPITAL
)

df = run_position_sizing_parallel(
    sizer=sizer, df=df, signals=signal_columns,
    chg_suffix="_chg1D_fx", sl_suffix="_stop_loss",
    close_col='close', n_jobs=-1, verbose=0
)

print("Position sizing and equity curves calculated")

Position sizing and equity curves calculated


In [8]:
# =============================================================================
# 7. RESULTS SUMMARY
# =============================================================================
print("=" * 70)
print(f"ANALYSIS SUMMARY - A2A.MI")
print("=" * 70)

# Performance metrics
abs_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
rel_return = (df['rclose'].iloc[-1] / df['rclose'].iloc[0] - 1) * 100

print(f"\n1. DATA:")
print(f"   Period: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"   Trading Days: {len(df):,}")

print(f"\n2. BUY & HOLD:")
print(f"   Absolute Return: {abs_return:+.2f}%")
print(f"   Relative Return (vs {BENCHMARK}): {rel_return:+.2f}%")

# Equity curve results
equity_cols = [col for col in df.columns if '_equity_equal' in col]
equity_results = []
for col in equity_cols:
    signal_name = col.replace('_equity_equal', '')
    final_equity = df[col].iloc[-1]
    total_return = (final_equity / INITIAL_CAPITAL - 1) * 100
    max_dd = (df[col].min() - df[col].max()) / df[col].max() * 100
    equity_results.append({
        'Signal': signal_name,
        'Final Equity': final_equity,
        'Return %': total_return,
        'Max DD %': max_dd
    })

equity_df = pd.DataFrame(equity_results).sort_values('Return %', ascending=False)

print(f"\n3. STRATEGY PERFORMANCE:")
print(equity_df.to_string(index=False))

# Best strategy
if len(equity_df) > 0:
    best = equity_df.iloc[0]
    print(f"\n4. BEST STRATEGY:")
    print(f"   Signal: {best['Signal']}")
    print(f"   Return: {best['Return %']:+.2f}%")
    print(f"   Final Equity: {best['Final Equity']:,.0f}")

print(f"\n5. DATAFRAME:")
print(f"   Shape: {df.shape}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print("\n" + "=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)

ANALYSIS SUMMARY - A2A.MI

1. DATA:
   Period: 2016-01-04 to 2026-02-05
   Trading Days: 2,565

2. BUY & HOLD:
   Absolute Return: +229.70%
   Relative Return (vs FTSEMIB.MI): +49.20%

3. STRATEGY PERFORMANCE:
       Signal  Final Equity  Return %  Max DD %
rsma_50100150   104169.7139  4.169714 -5.234648
rema_50100150   103036.4937  3.036494 -4.414268
       rbo_20   102938.2008  2.938201 -3.268990
     rtt_5020   101658.2808  1.658281 -2.630901
          rrg   101145.4793  1.145479 -3.428120
       rbo_50   100687.4900  0.687490 -3.067281
      rbo_100    97196.4997 -2.803500 -3.806519
   rema_50100    97080.1830 -2.919817 -4.188399
   rsma_50100    96042.9498 -3.957050 -5.211133
  rsma_100150    94470.2605 -5.529740 -5.869505
  rema_100150    91253.2852 -8.746715 -8.787135

4. BEST STRATEGY:
   Signal: rsma_50100150
   Return: +4.17%
   Final Equity: 104,170

5. DATAFRAME:
   Shape: (2565, 243)
   Memory: 4.76 MB

ANALYSIS COMPLETE


In [9]:
# =============================================================================
# 8. EXPORT RESULTS
# =============================================================================
import os

# Create output directory if needed
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

# Export equity summary
equity_df.to_csv(f"{output_dir}/{TICKER.replace('.', '_')}_equity.csv", index=False)

# Export latest signals (last row)
latest = df.tail(1)[['date'] + signal_columns].copy()
latest['ticker'] = TICKER
latest.to_csv(f"{output_dir}/{TICKER.replace('.', '_')}_signals.csv", index=False)

print(f"Results exported to {output_dir}/")
print(f"  - {TICKER.replace('.', '_')}_equity.csv")
print(f"  - {TICKER.replace('.', '_')}_signals.csv")

Results exported to ./results/
  - A2A_MI_equity.csv
  - A2A_MI_signals.csv
